In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time
import math

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sessConfig=tf.ConfigProto(gpu_options=gpu_options)


In [19]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 5

# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('/data/mnist', one_hot=True) 


Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [3]:
batch = mnist.train.next_batch( batch_size )

In [13]:
max( batch[1][1] )

1.0

In [8]:
batch[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
X = tf.placeholder( name="X", shape=[batch_size, 784], dtype=tf.float32 )
Y = tf.placeholder( name="Y", shape=[batch_size, 10], dtype=tf.float32 )

In [15]:
w = tf.Variable( tf.random_normal( shape=[784, 10], stddev=0.01 ), name = 'weights' )
b = tf.Variable( tf.zeros( shape=[1,10]), name = 'bias' )

In [18]:
logits = tf.matmul( X, w ) + b 
logits = tf.identity( logits, name = "logits" )

lossFunction = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits( logits = logits, labels = Y ) )
optimizer = tf.train.AdamOptimizer( learning_rate=learning_rate ).minimize( lossFunction ) 

In [ ]:
batches_per_epoch = math.ceil( mnist.train.num_examples / batch_size )
print( batches_per_epoch )

with tf.Session( config=sessConfig ) as sess:
    start_time = time.time()
    writer = tf.summary.FileWriter( "./graphs/logistic", sess.graph )
    sess.run( tf.global_variables_initializer() )
    
    for epoch in range( n_epochs ):
        print( "#Starting Epoch {0}".format( epoch + 1 ) )
        try:
            total_loss = 0.0
            i = 0
            for i in range( batches_per_epoch ) :
                if ( i %  math.ceil( ( batches_per_epoch / 10 ) ) == 0 ):
                    print( "----Starting batch {0}".format( i + 1 ) )
                #get batch
                batch = mnist.train.next_batch( batch_size )
                #run batch
                _,batchLoss = sess.run( [optimizer, lossFunction], feed_dict = { X: batch[0], Y: batch[1] } )
                total_loss += batchLoss
        except tf.errors.OutOfRangeError:
            print( "Epoch {0}: {1}".format( epoch, math.sqrt( total_loss / mnist.train.num_examples ) ) )
            pass
    
    writer.flush()
    writer.close()
    print('Total time: {0} seconds'.format(time.time() - start_time))   
    
    w_out, b_out = sess.run( [w,b] )
    
    print('Optimization Finished!') # should be around 0.35 after 25 epochs

430
